In [1]:
import pandas as pd

# --- Part 1: Load Your Datasets ---

try:
    # Load the completed company data from Week 3
    company_data = pd.read_csv('companies_with_full_emissions_data.csv')
    print("✅ Loaded complete company emissions data.")

    # Load the user's portfolio
    portfolio_df = pd.read_csv('my_portfolio.csv')
    print("✅ Loaded user portfolio data.")

except FileNotFoundError as e:
    print(f"❌ Error: Could not find a required file. Make sure '{e.filename}' is in your project folder.")
    exit()

# --- Part 2: Merge Portfolio with Company Data ---

# We only need the most recent data for each company.
# Let's sort by year and drop older duplicates to get the latest stats.
company_data_latest = company_data.sort_values('Year').drop_duplicates('CompanyName', keep='last')

# Merge the portfolio with the company data based on the 'CompanyName'
# This adds financial and emissions data to each of our holdings.
portfolio_complete = pd.merge(
    portfolio_df,
    company_data_latest,
    on='CompanyName',
    how='left' # Use a 'left' merge to keep all portfolio entries, even if a company isn't found
)

print("\n--- Merged Portfolio Details ---")
print(portfolio_complete)


# --- Part 3: Calculate Financed Emissions ---

# Handle cases where a company in the portfolio was not found in our dataset
portfolio_complete.dropna(subset=['MarketCap', 'CarbonEmissions'], inplace=True)

# Calculate the 'Ownership %' based on the investment and the company's market cap
portfolio_complete['OwnershipFraction'] = portfolio_complete['Investment'] / portfolio_complete['MarketCap']

# Calculate the financed emissions for each holding
portfolio_complete['FinancedEmissions'] = portfolio_complete['OwnershipFraction'] * portfolio_complete['CarbonEmissions']

# Sum up the emissions from all holdings to get the total portfolio footprint
total_portfolio_emissions = portfolio_complete['FinancedEmissions'].sum()


# --- Part 4: Display the Final Report ---

print("\n--- Final Calculation Breakdown ---")
# Display only the most important columns for the final report
print(portfolio_complete[['CompanyName', 'Investment', 'MarketCap', 'CarbonEmissions', 'FinancedEmissions']])

print("\n" + "="*50)
print(f"🌍 TOTAL PORTFOLIO CARBON FOOTPRINT 🌍")
print("="*50)
# The ':,.2f' formats the number nicely with commas and 2 decimal places.
print(f"\n>>> {total_portfolio_emissions:,.2f} tons of CO₂e")

✅ Loaded complete company emissions data.
✅ Loaded user portfolio data.

--- Merged Portfolio Details ---
  CompanyName  Investment  CompanyID       Industry         Region  Year  \
0   Company_1       50000          1         Retail  Latin America  2025   
1  Company_10      150000         10     Technology  Latin America  2025   
2  Company_22       75000         22  Manufacturing  Latin America  2025   
3   Company_4      200000          4     Technology         Africa  2025   

    Revenue  ProfitMargin  MarketCap  GrowthRate  ESG_Overall  \
0     734.5           5.0      628.8         6.9         59.2   
1  104787.3          17.1   412517.1       -11.2         69.6   
2    1009.7          11.8     2210.1         5.9         60.1   
3    7675.6          22.9    42400.4        17.1         64.1   

   ESG_Environmental  ESG_Social  ESG_Governance  CarbonEmissions  WaterUsage  \
0               70.7        29.1            78.0    -5.154505e+04     25902.9   
1               80.9     